## Obectives

1. LR, SVM, MNB with Gridsearch selected parameters
2. Features BOW,TFIDF,w2v
3. Can easily add new feature or model by creating new function


## TODO

1. Bagging model, Boosting model
2. Word embedding
3. Gridsearch again for ngrams for char 
4. explore more paremeter for countvectorizer and TFIDF

## Set up Neptune for logging

In [ ]:
import neptune
from neptunecontrib.monitoring.metrics import expand_prediction, log_class_metrics, log_binary_classification_metrics, log_classification_report,log_confusion_matrix,log_prediction_distribution
from neptunecontrib.api import log_table
import os
from dotenv import load_dotenv

load_dotenv()
NEPTUNE_PROJECT= os.getenv('NEPTUNE_PROJECT')
NEPTUNE_API_TOKEN = os.getenv(('NEPTUNE_API_TOKEN'))
neptune.init(project_qualified_name= NEPTUNE_PROJECT,api_token=NEPTUNE_API_TOKEN) 
             

## Load necessary modules

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
#from sklearn.metrics import accuracy_score, f1_score, precision_score,roc_curve,roc_auc_score,confusion_matrix,recall_score
from sklearn.pipeline import Pipeline
import re
import gensim

# import helper function script
import sys
sys.path.insert(1,'G:\\Github\\Sinhala-Hate-Speech-Detection')
import utills

In [5]:
# load datasets
path = '../Datasets/processed/no_preprocessing/'
df_A = pd.read_csv(path+'df_A.csv')    
df_B = pd.read_csv(path+'df_B.csv')    # fb dataset -kaggle
df_C = pd.read_csv(path+'df_C.csv') 
df_D = pd.read_csv(path+'df_D.csv') 
df_A_B = pd.read_csv(path+'df_A_B.csv') 
df_B_C_D = pd.read_csv(path+'df_B_C_D.csv') 
df_A_B_C = pd.read_csv(path+'df_A_B_C.csv') 
df_A_C_D = pd.read_csv(path+'df_A_C_D.csv') 
df_A_B_D = pd.read_csv(path+'df_A_B_D.csv') 
df_all = pd.read_csv(path+'df_all.csv') 

In [106]:
def classifier_feature(datasets,models,features):
    final_result =pd.DataFrame(columns=['Accuracy','F1-score','Recall','Precision','AUC'])
    for df_name,df in datasets.items():
        df_result =pd.DataFrame(columns=['Accuracy','F1-score','Recall','Precision','AUC'])
        X_train,X_test,Y_train,Y_test = utills.prepare_dataset(df,df_name)
        for feature_name,feature in features.items():
            feature_result =pd.DataFrame(columns=['Accuracy','F1-score','Recall','Precision','AUC'])
            X_train_f,X_test_f = feature(X_train,X_test)
            for model_name,model in models.items():
                name = df_name + "+" + feature_name+ "+"+ model_name
                print(name)
                Y_pred = model(X_train_f,X_test_f,Y_train)
                accuracy, f1_score, recall, precision, auc = result(Y_test,Y_pred)
                final_result.loc[name] = [accuracy, f1_score, recall, precision, auc]
                feature_result.loc[model_name] =[accuracy, f1_score, recall, precision, auc]
                key =model_name + "+"+ feature_name
                df_result.loc[key] = [accuracy, f1_score, recall, precision, auc]
                log_result(Y_test,Y_pred,name,df_name,feature_name,model_name)
            print(" ==== ",feature_name ," ==== ")
            display(feature_result)
            log_table(feature_name,feature_result)
        print(" ==== ",df_name ," ==== ")
        display(df_result)
        log_table(df_name,df_result)
    display(final_result)
    log_table(name,final_result)



In [107]:
# load datasets
df_dict = {'df_A': df_A,'df_B': df_B,'df_C' :df_C,'df_D' :df_D,'df_A_B' :df_A_B,'df_A_C_D':df_A_C_D,'df_A_B_C':df_A_B_C,'df_A_B_D':df_A_B_D,'df_B_C_D':df_B_C_D,'df_all':df_all}
# feature dict
feature_dict={"bow_word":bow_word,"bow_char":bow_char,"tfidf_word":tfidf_word,"tfidf_char":tfidf_char}
# model dict
model_dict = {"LR":lr,"SVC":svc,"MNB":MNB}
classifier_feature(df_dict, model_dict, feature_dict)

df_A 6468
X train (4527,) Y train (4527,) X test (1941,) Y test (1941,)


C:\Users\Kavishka\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


df_A+bow_word+LR
df_A+bow_word+SVC


KeyboardInterrupt: 

In [101]:
# feature funtions

# bow - word
def bow_word(X_train,X_test):
    bow = CountVectorizer(analyzer="word", tokenizer=lambda text: text.split(),ngram_range=(1,2),lowercase=False)
    bow.fit(X_train)
    X_train_bow = bow.transform(X_train)
    X_test_bow = bow.transform(X_test)

    #print(bow.get_feature_names()[:20])
    #print('The shape is', bow.shape)
    # postion
    #print(bow.vocabulary_)

    return X_train_bow,X_test_bow

# bow - char
def bow_char(X_train,X_test):
    bow = CountVectorizer(analyzer="char",ngram_range=(2,4),lowercase=False)
    bow.fit(X_train)
    X_train_bow = bow.transform(X_train)
    X_test_bow = bow.transform(X_test)
    # X_train_bow = bow.fit_transform(X_train)
    # #X_train_bow = bow.transform(X_train)
    # X_test_bow = bow.transform(X_test)
    #print(bow.get_feature_names()[:20])
    #print('The shape is', tfidf.shape)
    # postion
    #print(bow.vocabulary_)

    return X_train_bow,X_test_bow

# TFIDF - word
def tfidf_word(X_train,X_test):
    tfidf = TfidfVectorizer(analyzer="word",  tokenizer=lambda text: text.split(),ngram_range=(1,2),lowercase=False)
    X_train_tfidf = tfidf.fit_transform(X_train)
    #print(X_train_tfidf.shape)
    #X_train_tfidf = tfidf.transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)
    # print(tfidf.get_feature_names()[:20])
    # #print('The shape is', tfidf.shape)
    # # postion
    # print(tfidf.vocabulary_)
    return X_train_tfidf,X_test_tfidf

# TFIDF - char
def tfidf_char(X_train,X_test):
    tfidf = TfidfVectorizer(analyzer="char",ngram_range=(2,4),lowercase=False)
    X_train_tfidf  = tfidf.fit_transform(X_train)
    #X_train_tfidf = tfidf.transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)

    # print(tfidf.get_feature_names()[:20])
    # #print('The shape is', tfidf.shape)
    # # postion
    # print(tfidf.vocabulary_)
    
    return X_train_tfidf,X_test_tfidf

# word2vec




# fasttext

In [100]:
#  model functions
# lr
def lr(X_train,X_test,Y_train):
    lr = LogisticRegression(C=10,max_iter=350)
    lr.fit(X_train,Y_train)
    Y_pred = lr.predict(X_test)
    return Y_pred


# SVC
def svc(X_train,X_test,Y_train):
    svc = SVC(kernel = "linear")
    svc.fit(X_train,Y_train)
    Y_pred = svc.predict(X_test)
    return Y_pred

# MNB
def MNB(X_train,X_test,Y_train):
    nb = MultinomialNB(alpha =0.01)
    nb.fit(X_train,Y_train)
    Y_pred = nb.predict(X_test)
    Y_prob = nb.predict_proba(X_test)[:,1]
    return Y_pred

# XGBoost
# .fit(X_train,Y_train)
# Y_pred = .predict(X_test)
# return Y_pred


# Randomfroest

In [ ]:

def dataset_check(df_list,vectorizer,feature_name,model,model_name):
    
    final_result =pd.DataFrame(columns=['Accuracy','F1-score','Recall','Precision','AUC'])
    for name,df in df_list.items():
        X_train,X_test,Y_train,Y_test = utills.prepare_dataset(df,name)
        model_pipe, Y_pred = model(X_train,X_test,Y_train,vectorizer,feature_name)
        # gridsearch function and get best parameter for dataset
        gridsearch(model_pipe,param_grid,X_train,Y_train)
        final_result = log_result(Y_test,Y_pred,feature_name,name,final_result,model_name)
         
    
    return final_result
      


In [104]:
def result(y_test, y_pred):
    import sklearn.metrics as metrics
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    classes = np.unique(y_test)
    #y_test_array = pd.get_dummies(y_test, drop_first=False).values
    ## Accuracy, Precision, Recall
    accuracy = metrics.accuracy_score(y_test, y_pred)
    auc = metrics.roc_auc_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
#     print("Accuracy:",  round(accuracy, 2))
#     print("Auc:", round(auc, 2))
#     print("Precsion:", round(precision, 2))
#     print("f1_score:", round(f1_score, 2))
#     print("recall:", round(recall, 2))
#     print("Detail:")
    #print(metrics.classification_report(y_test, y_pred))

    # Plot confusion matrix
#     cm = metrics.confusion_matrix(y_test, y_pred)
#     fig, ax = plt.subplots()
#     sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap=plt.cm.Blues, cbar=False)
#     ax.set(xlabel="Predicted", ylabel="Actual", xticklabels=classes,
#            yticklabels=classes, title="Confusion matrix")
#     plt.yticks(rotation=0)
#     plt.show()
#     utills.PlotRocAuc(y_test, y_pred, 'green', 'LR')
  
    return (accuracy, f1_score, recall, precision, auc)

In [21]:
def log_result(Y_test,Y_pred,name,df_name,feature_name,model_name):
    
    
    print("========= Eperiment - ",name," =========")
    neptune.create_experiment(name)
    neptune.append_tag(['Dataset experiment 3',df_name,feature_name,model_name,name])
    
    log_class_metrics(Y_test, Y_pred)
    log_confusion_matrix(Y_test, Y_pred)
    log_classification_report(Y_test, Y_pred)

    #accuracy,f1_score,recall,precision,auc = result(Y_test,Y_pred)
    
    # neptune.log_metric('acc', accuracy)
    # neptune.log_metric('auc', auc)
    # neptune.log_metric('recall', recall)
    # neptune.log_metric('precison', precision)
    # neptune.log_metric('f1_score', f1_score)
    
    
    #final_result.loc[df_name] = [accuracy,f1_score,recall,precision,auc]
    
    #return final_result

In [ ]:
# Loading pretrained FastText word embeddings in Sinhala
import fasttext
import fasttext.util
ft = fasttext.load_model('/content/cc.si.300.bin')
ft.get_dimension()
# Mapping FastText word vectors with word in the dataset 
embeddings_matrix = np.zeros((vocab_size+1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = ft.get_word_vector(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector